Import 

In [166]:
import cv2
import os
import numpy as np              
from skimage.io import imread, imsave
from tqdm import tqdm
from Levenshtein import distance as levenshtein_distance


from lsb import psnr, batch_embed, batch_extract


Deklarasi Gaussian Noise

In [167]:
def add_gaussian_noise(image, mean=0, std=10):
    noise = np.random.normal(mean, std, image.shape)
    noisy_img = image + noise
    return np.clip(noisy_img, 0, 255).astype(np.uint8)

Batch Attack

In [168]:
def batch_attack_gaussian_multi(input_dir, base_output_dir="noise_gaussian_batik", std_values=[1, 5, 10], mean=0):
    os.makedirs(base_output_dir, exist_ok=True)
    for std in std_values:
        subfolder = os.path.join(base_output_dir, f"std{std}")
        os.makedirs(subfolder, exist_ok=True)
        print(f"\n>== Membuat noise std={std} ke folder: {subfolder} ==<")
        
        for fname in os.listdir(input_dir):
            if fname.lower().endswith(('.png', '.jpg', '.jpeg')):
                img = imread(os.path.join(input_dir, fname))
                noisy = add_gaussian_noise(img, mean, std)
                imsave(os.path.join(subfolder, fname), noisy)
        
        print(f"✓ Noise std={std} selesai → {subfolder}")

Text to Bits Vice Versa

In [169]:
def text_to_bits(text):
    return ''.join(format(ord(c), '08b') for c in text)

def bits_to_text(bits):
    chars = []
    for i in range(0, len(bits), 8):
        byte = bits[i:i+8]
        chars.append(chr(int(byte, 2)))
    return ''.join(chars)

Bit Error

In [170]:
def bit_error_rate(original_text, extracted_text):
    orig_bits = text_to_bits(original_text)
    extr_bits = text_to_bits(extracted_text)
    min_len = min(len(orig_bits), len(extr_bits))
    errors = sum(1 for i in range(min_len) if orig_bits[i] != extr_bits[i])
    total = len(orig_bits)
    return errors / total if total > 0 else 0

Uji Robustness

In [171]:
#Beberapa Standart Deviasi untuk diuji
std_values = [5 , 10]
results = []
original_message = (
    "Batik merupakan salah satu warisan budaya Indonesia yang memiliki nilai sejarah dan filosofi yang sangat tinggi. "
    "Setiap motif batik tidak hanya menghadirkan keindahan visual, tetapi juga membawa pesan dan makna yang mendalam. "
    "Motif parang, misalnya, menggambarkan kekuatan, semangat pantang menyerah, serta keberanian dalam menghadapi tantangan hidup. "
    "Sementara itu, motif kawung dipercaya melambangkan kesucian hati dan keinginan manusia untuk mencapai keseimbangan hidup."
)

# Uji Batch Atk dgn std values
for std in std_values:
    print(f"\n>== Serangan Gaussian std={std} ==<")
    noisy_dir = f"noise_gaussian_batik/std{std}"
    batch_attack_gaussian("stego_batik", noisy_dir, mean=0, std=std)
    res_noisy = batch_extract("dataset_png", noisy_dir)

    success = 0
    psnr_vals = []
    ber_vals = []

    for fname, msg, psnr_val in res_noisy:
        ber = bit_error_rate(original_message, msg)
        ber_vals.append(ber)
        if "Batik" in msg:
            success += 1
        if psnr_val is not None:
            psnr_vals.append(psnr_val)

        print(f"\nFile: {fname}")
        print(f"PSNR: {psnr_val:.2f} dB")
        print(f"Bit Error Rate: {ber:.6f}")
        
        snippet = msg[:200] + "..." if len(msg) > 200 else msg
        print("Extracted snippet:")
        print("-" * 60)
        print(snippet)
        print("-" * 60)

        

    avg_psnr = np.mean(psnr_vals) if psnr_vals else 0
    avg_ber = np.mean(ber_vals) if ber_vals else 1

    print(f"\nRangkuman std={std}:")
    print(f"  Berhasil ekstraksi: {success}/{len(res_noisy)}")
    print(f"  Rata-rata PSNR : {avg_psnr:.2f} dB")
    print(f"  Rata-rata BER  : {avg_ber:.6f}")
    results.append((std, avg_psnr, avg_ber, success))
    print("=" * 60)
    


>== Serangan Gaussian std=5 ==<
Noise attack telah selesai untuk std=5 → folder: noise_gaussian_batik/std5

File: stego_1.png
PSNR: 34.21 dB
Bit Error Rate: 0.012156
Extracted snippet:
------------------------------------------------------------
(Invalid payload)
------------------------------------------------------------

File: stego_10.png
PSNR: 34.55 dB
Bit Error Rate: 0.012156
Extracted snippet:
------------------------------------------------------------
(Invalid payload)
------------------------------------------------------------

File: stego_11.png
PSNR: 34.42 dB
Bit Error Rate: 0.012156
Extracted snippet:
------------------------------------------------------------
(Invalid payload)
------------------------------------------------------------

File: stego_12.png
PSNR: 34.22 dB
Bit Error Rate: 0.012156
Extracted snippet:
------------------------------------------------------------
(Invalid payload)
------------------------------------------------------------

File: stego_13.p

Hasil ekstraksi

In [172]:
print("\n===== HASIL AKHIR UJI ROBUSTNESS (Gaussian Noise) =====")
print(f"{'STD':<10}{'Avg PSNR (dB)':<20}{'Avg BER':<15}{'Berhasil Ekstraksi':<20}")

for std, avg_psnr, avg_ber, success in results:
    print(f"{std:<10}{avg_psnr:<20.2f}{avg_ber:<15.6f}{success:<20}")


===== HASIL AKHIR UJI ROBUSTNESS (Gaussian Noise) =====
STD       Avg PSNR (dB)       Avg BER        Berhasil Ekstraksi  
5         34.36               0.012156       0                   
10        30.48               0.011929       0                   
